In [2]:
import torch
import numpy as np
from torchvision.models import resnet50
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_cifar10, load_cifar100
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_dataloaders, test_dataloaders, test_dataloader_all = load_cifar10()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
